<a href="https://colab.research.google.com/github/akshat-giri/Deep-Learning/blob/master/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create CNN Model and Optimize it using Keras Tuner

In [0]:
import pandas as pd

In [0]:
import numpy as np

In [9]:
%tensorflow_version

Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


## Downloading Keras Tuner

In [8]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=501ecb41878907ba1771dcf084251a5216518c62da20c30a2b30fd878e83cf85
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=c71509221b9141a37a17cc7b3f5c7288107740e7f1b943bb07394f1bf8156483
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras

In [11]:
print(tf.__version__)


2.2.0-rc3


## Loading Fashion MNIST datasets

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

In [13]:
(train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


## Scaling Our Data

In [0]:
train_images = train_images/255.0
test_images = test_images/255.0

In [15]:
train_images[0].shape

(28, 28)

In [16]:
train_images

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [0]:
# reshaping our images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

## Creating a model Builder

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [24]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4347 - accuracy: 0.8453 - val_loss: 0.3113 - val_accuracy: 0.8815
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2898 - accuracy: 0.8944 - val_loss: 0.2784 - val_accuracy: 0.8955
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2459 - accuracy: 0.9088 - val_loss: 0.2700 - val_accuracy: 0.9005


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3963 - accuracy: 0.8558 - val_loss: 0.2941 - val_accuracy: 0.8910
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2605 - accuracy: 0.9026 - val_loss: 0.2628 - val_accuracy: 0.9035
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2048 - accuracy: 0.9227 - val_loss: 0.2460 - val_accuracy: 0.9098


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.5407 - accuracy: 0.8146 - val_loss: 0.4526 - val_accuracy: 0.8382
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3952 - accuracy: 0.8551 - val_loss: 0.3941 - val_accuracy: 0.8570
Epoch 3/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3787 - accuracy: 0.8601 - val_loss: 0.3803 - val_accuracy: 0.8618


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4544 - accuracy: 0.8366 - val_loss: 0.3316 - val_accuracy: 0.8785
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3270 - accuracy: 0.8801 - val_loss: 0.3449 - val_accuracy: 0.8753
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2852 - accuracy: 0.8937 - val_loss: 0.3325 - val_accuracy: 0.8797


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4102 - accuracy: 0.8542 - val_loss: 0.3274 - val_accuracy: 0.8782
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2635 - accuracy: 0.9051 - val_loss: 0.2609 - val_accuracy: 0.9060
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2095 - accuracy: 0.9235 - val_loss: 0.2587 - val_accuracy: 0.9072


INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,290,010
Trainable params: 2,290,010
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1599 - accuracy: 0.9395 - val_loss: 0.2398 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1196 - accuracy: 0.9543 - val_loss: 0.2708 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0905 - accuracy: 0.9660 - val_loss: 0.2978 - val_accuracy: 0.9107
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0693 - accuracy: 0.9750 - val_loss: 0.3256 - val_accuracy: 0.9122
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0534 - accuracy: 0.9797 - val_loss: 0.3905 - val_accuracy: 0.9085
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0433 - accuracy: 0.9847 - val_loss: 0.4434 - val_accuracy: 0.9050
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0330 - accuracy: 0.9877 - val_loss: 0.4992 - val_accuracy

### We have trained our model with 98.77% accuracy